If you have the ERA5 data, wave analysis file, and model data collected in one directory, and named as described below, these notebooks should be able to replicate Figures 1–4 in Bartusek et al., 2022, Nature Climate Change. This is the notebook to import most of the data and do most of the analysis that's shared between the figures. The other notebooks each have some extra computation but require this notebook to be run first. They can either each use this notebook's kernel, if using JupyterLab, or each individually run this notebook in their own kernels (via %run ./All_figs_data_setup.ipynb in their first code cell). Take care to read the "External data necessary..." cell below.

--Sam Bartusek, 2022--

# Imports / functions

In [1]:
# Imports:

import glob
import re
import numpy as np
from numpy import diff
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cftime
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib_inline
import matplotlib.cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import matplotlib.ticker as mticker
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import datetime
import time
import scipy
import scipy.stats as ss
import scipy.signal
from scipy.interpolate import griddata
from scipy.signal import argrelmin
from scipy.ndimage import interpolation
from scipy.stats import genextreme as gev
import scipy.stats.mstats as mstats
import math
from math import log10, floor
import os
import dask.diagnostics as dd
import dask.array as da
from dask.diagnostics import ProgressBar
pbar = ProgressBar() 
import seaborn as sns
from sklearn import linear_model
import cmasher as cmr
import climextremes  # Will take some time to import; you also must have R installed for it to work on top of
from climextremes import fit_gev
from climextremes import calc_riskRatio_gev
import pickle


# Functions:

# https://github.com/darothen/plot-all-in-ncfile/blob/master/plot_util.py
def cyclic_dataarray(da, coord='longitude'):
    """ Add a cyclic coordinate point to a DataArray along a specified
    named coordinate dimension.
    """
    assert isinstance(da, xr.DataArray)

    lon_idx = da.dims.index(coord)
    cyclic_data, cyclic_coord = add_cyclic_point(da.values,
                                                 coord=da.coords[coord],
                                                 axis=lon_idx)

    # Copy and add the cyclic coordinate and data
    new_coords = dict(da.coords)
    new_coords[coord] = cyclic_coord
    new_values = cyclic_data

    new_da = xr.DataArray(new_values, dims=da.dims, coords=new_coords)

    # Copy the attributes for the re-constructed data and coords
    for att, val in da.attrs.items():
        new_da.attrs[att] = val
    for c in da.coords:
        for att in da.coords[c].attrs:
            new_da.coords[c].attrs[att] = da.coords[c].attrs[att]

    return new_da

# http://hrishichandanpurkar.blogspot.com/2017/09/vectorized-functions-for-correlation.html
def lag_linregress_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three 
    dimensions (time,lat,lon). 
    Datasets can be provided in any order, but note that the regression slope 
    and intercept will be calculated for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, 
    and standard error on regression between the two datasets along their 
    aligned time dimension.  
    Lag values can be assigned to either of the data, with lagx shifting x, and
    lagy shifting y, with the specified lag amount. 
    """ 
    #1. Ensure that the data are properly alinged to each other. 
    x,y = xr.align(x,y)

    #2. Add lag information if any, and shift the data accordingly
    if lagx!=0:

        # If x lags y by 1, x must be shifted 1 step backwards. 
        # But as the 'zero-th' value is nonexistant, xr assigns it as invalid 
        # (nan). Hence it needs to be dropped
        x   = x.shift(time = -lagx).dropna(dim='time')

        # Next important step is to re-align the two datasets so that y adjusts
        # to the changed coordinates of x
        x,y = xr.align(x,y)

    if lagy!=0:
        y   = y.shift(time = -lagy).dropna(dim='time')
        x,y = xr.align(x,y)

    #3. Compute data length, mean and standard deviation along time axis: 
    n = y.notnull().sum(dim='time')
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd  = x.std(axis=0)
    ystd  = y.std(axis=0)

    #4. Compute covariance along time axis
    cov   =  np.sum((x - xmean)*(y - ymean), axis=0)/(n)

    #5. Compute correlation along time axis
    cor   = cov/(xstd*ystd)

    #6. Compute regression slope and intercept:
    slope     = cov/(xstd**2)
    intercept = ymean - xmean*slope  

    #7. Compute P-value and standard error
    #Compute t-statistics
    tstats = cor*np.sqrt(n-2)/np.sqrt(1-cor**2)
    stderr = slope/tstats

    from scipy.stats import t
    pval   = t.sf(tstats, n-2)*2
    pval   = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    return cov,cor,slope,intercept,pval,stderr

states_provinces_50 = cartopy.feature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')


/tmp/ipykernel_12822/2460229006.py:25: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')
R[write to console]: Error in library(devtools) : there is no package called ‘devtools’
Calls: <Anonymous> -> <Anonymous> -> library



Current version: 0.2.1 does not match requested version: 0.2.2. Attempting installation of R climextRemes package (this may take a few minutes) ...


R[write to console]: also installing the dependencies ‘gert’, ‘ragg’, ‘usethis’, ‘pkgdown’


R[write to console]: trying URL 'https://cran.us.r-project.org/src/contrib/gert_1.9.0.tar.gz'

R[write to console]: Content type 'application/octet-stream'
R[write to console]:  length 121010 bytes (118 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write t

Found OpenSSL3
Using static libgit2 for Linux x86_64
Using PKG_CFLAGS=-DSTATIC_LIBGIT2 -I/tmp/Rtmp1I9hhm/R.INSTALL32857210c8f8/gert/libgit2-1.4.2-x86_64_linux/include
Using PKG_LIBS=-L/tmp/Rtmp1I9hhm/R.INSTALL32857210c8f8/gert/libgit2-1.4.2-x86_64_linux/lib -lgit2 -lrt -lpthread -lssh2 -lssl -lcrypto -ldl
Configuration OK!
rm -f gert.so branch.o clone.o commit.o config.o conflicts.o files.o init.o merge.o rebase.o stash.o submodules.o tag.o utils.o version.o
x86_64-conda-linux-gnu-cc -I"/home/samuelb/.conda/envs/r_py_env/lib/R/include" -DNDEBUG -DSTATIC_LIBGIT2 -I/tmp/Rtmp1I9hhm/R.INSTALL32857210c8f8/gert/libgit2-1.4.2-x86_64_linux/include -DR_NO_REMAP -DSTRICT_R_HEADERS  -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/samuelb/.conda/envs/r_py_env/include -I/home/samuelb/.conda/envs/r_py_env/include -Wl,-rpath-link,/home/samuelb/.conda/envs/r_py_env/lib  -fvisibility=hidden -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-secti

** libs


...

In file included from ppm_dev.cpp:4:
AggDevicePpm.h: In member function 'bool AggDevicePpm<PIXFMT>::savePage() [with PIXFMT = agg::pixfmt_alpha_blend_rgb<agg::blender_rgb_pre<agg::rgba8T<agg::linear>, agg::order_rgb>, agg::row_accessor<unsigned char>, 3>]':
AggDevicePpm.h:23:13: warning: ignoring return value of 'size_t fwrite(const void*, size_t, size_t, FILE*)', declared with attribute warn_unused_result [-Wunused-result]
   23 |       fwrite(this->buffer, 1,this-> width * this->height * this->bytes_per_pixel, fd);
      |       ~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


x86_64-conda-linux-gnu-c++ -std=gnu++11 -I"/home/samuelb/.conda/envs/r_py_env/lib/R/include" -DNDEBUG  -I'/home/samuelb/.conda/envs/r_py_env/lib/R/library/systemfonts/include' -I'/home/samuelb/.conda/envs/r_py_env/lib/R/library/textshaping/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/samuelb/.conda/envs/r_py_env/include -I/home/samuelb/.conda/envs/r_py_env/include -Wl,-rpath-link,/home/samuelb/.conda/envs/r_py_env/lib  -I./agg/include -I/usr/include/freetype2 -I/usr/include/libpng15   -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/samuelb/.conda/envs/r_py_env/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1639563404388/work=/usr/local/src/conda/r-base-4.1.2 -fdebug-prefix-map=/home/samuelb/.conda/envs/r_py_env=/usr/local/src/conda-prefix  -c tiff_dev.cpp -o tiff_dev.o
x86_64-conda-linux-gnu-c++ -std=gn

/home/samuelb/.conda/envs/r_py_env/bin/../lib/gcc/x86_64-conda-linux-gnu/9.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -lpng15
collect2: error: ld returned 1 exit status
make: *** [/home/samuelb/.conda/envs/r_py_env/lib/R/share/make/shlib.mk:10: ragg.so] Error 1
ERROR: compilation failed for package ‘ragg’
* removing ‘/home/samuelb/.conda/envs/r_py_env/lib/R/library/ragg’
ERROR: dependency ‘gert’ is not available for package ‘usethis’
* removing ‘/home/samuelb/.conda/envs/r_py_env/lib/R/library/usethis’
ERROR: dependency ‘ragg’ is not available for package ‘pkgdown’
* removing ‘/home/samuelb/.conda/envs/r_py_env/lib/R/library/pkgdown’
ERROR: dependencies ‘usethis’, ‘pkgdown’ are not available for package ‘devtools’
* removing ‘/home/samuelb/.conda/envs/r_py_env/lib/R/library/devtools’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/RtmptQr6Xn/downloaded_packages’
R[write to console]: 
R[write to console]: 

R

Unable to install R package devtools; needed for version-specific installation of R climextRemes.


/home/samuelb/.conda/envs/r_py_env/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


# Locate data

In [2]:
# External data necessary to replicate the 4 main figures and how to name the files in order to be recognized by these notebooks:

## ERA5 land mask     #### NAME: 'era5_lsm.nc'

## 6-hourly ERA5 data for June–July 2021 over Northern Hemisphere:     #### NAMES: 'ERA5*2021*nh.nc' (these notebooks call them with the stars, so you may put whatever you want in place of the stars)
### 2 meter temperature
### Skin temperature
### 500 hPa geopotential height
### 0–7 cm volumetric soil moisture
### 300 hPa zonal wind
### 300 hPa meridional wind
### Outgoing longwave radiation

## 6-hourly ERA5 data for June–July 1979–2020 over Northern Hemisphere:     #### NAMES: 'ERA5*1979_2020*6hr_nh.nc'
### Same variables as above

## 6-hourly mean climatology for June–July 1981–2010 for each variable (using CDO)     #### NAMES: 'ERA5*19812010clim*nh.nc'
### Same variables as above

## 1-monthly ERA5 skin temperature anomalies since 1959 covering 40–60N, 130–110W (with 1981–2010 mean subtracted using CDO):     #### NAME: 'ERA_ts_pnw_1959-2021_1month_anom.nc'

## Same as above but 2 meter temperature:     #### NAME: 't2m_1950-2021_1d_noleap_anom_junjulaug_PNW.nc'

## Panetary wave analysis using 300hPa zonal and meridional wind     #### NAME: 'wave_2021.csv'

## Model data:
### See https://doi.org/10.5281/zenodo.5800726:     #### NAMES: See Fig_3.ipynb


# Set datapath containing all these files:
path = '/dx09/samuelb/pnw_hw/'


# Import/prepare data

In [3]:
## IMPORT/PREPARE ERA5 DATA 2021 ## 


# Calculate 2021 anomalies of ERA5 data

landmask = xr.open_dataset(path + 'era5_lsm.nc',decode_times=False).lsm.isel(time=0).drop('time')

raw = xr.open_mfdataset(glob.glob(path + 'ERA5*2021*nh.nc'),combine='by_coords')
raw['t2m'] = raw.t2m - 273.15
raw['t2mland'] = raw.t2m.where(landmask>.5)
raw['skt'] = raw.skt - 273.15
raw['sktland'] = raw.skt.where(landmask>.5)
raw['z'] = raw.z / 9.81
raw['swvl1'] = raw.swvl1.where(landmask>.5)
raw['speed'] = (raw.u**2 + raw.v**2)**(1/2)
x,y = np.meshgrid(raw.longitude,raw.latitude)

clim = xr.open_mfdataset(glob.glob(path + 'ERA5*19812010clim*6hr_nh.nc'),combine='by_coords')
clim['t2m'] = clim.t2m - 273.15
clim['t2mland'] = clim.t2m.where(landmask>.5)
clim['skt'] = clim.skt - 273.15
clim['sktland'] = clim.skt.where(landmask>.5)
clim['z'] = clim.z / 9.81
clim['swvl1'] = clim.swvl1.where(landmask>.5)
clim['speed'] = (clim.u**2 + clim.v**2)**(1/2)
clim = clim.isel(time=slice(0,len(raw.time))).assign(time = raw['time']).drop('time_bnds')

anom = raw - clim
anom['speed'] = (anom.u**2 + anom.v**2)**(1/2)


# Convert to daily

rawdaily = raw.sel(time=anom.time).resample(time='1D').mean('time').compute()
climdaily = clim.resample(time='1D').mean('time').compute()
anomdaily = anom.resample(time='1D').mean('time').compute()


# Average throughout 2021 heatwave

timeslice = slice('2021-06-25','2021-07-03')

rawslice = rawdaily.sel(time=timeslice).mean('time').compute()
climslice = climdaily.sel(time=timeslice).mean('time').compute()
anomslice = anomdaily.sel(time=timeslice).mean('time').compute()


# Take weighted spatial mean over PNW

pnwlat = slice(60,40)
pnwlon = slice(-130,-110)
weights = np.cos(np.deg2rad(anom.latitude))
weights.name = "weights"

raw_pnw = raw.sel(latitude=pnwlat,longitude=pnwlon)
raw_pnw_mean = raw_pnw.weighted(weights).mean(('latitude','longitude')).compute()
clim_pnw = clim.sel(latitude=pnwlat,longitude=pnwlon)
clim_pnw_mean = clim_pnw.weighted(weights).mean(('latitude','longitude')).compute()
anom_pnw = anom.sel(latitude=pnwlat,longitude=pnwlon)
anom_pnw_zonal = anom_pnw.mean('longitude').compute()
anom_pnw_merid = anom_pnw.weighted(weights).mean('latitude').compute()
anom_pnw_mean = anom_pnw.weighted(weights).mean(('latitude','longitude')).compute()
anomdaily_pnw_mean = anomdaily.sel(latitude=pnwlat,longitude=pnwlon).weighted(weights).mean(('latitude','longitude')).compute()


# Average throughout June 2021

raw_jun = raw.sel(time='2021-06').mean('time').compute()
raw_jun_pnw_mean = raw_jun.sel(latitude=pnwlat,longitude=pnwlon).weighted(weights).mean(('latitude','longitude')).compute()

clim_jun = climdaily.sel(time='2021-06').mean('time').compute()
clim_jun_pnw_mean = clim_jun.sel(latitude=pnwlat,longitude=pnwlon).weighted(weights).mean(('latitude','longitude')).compute()

anom_jun = anomdaily.sel(time='2021-06').mean('time').compute()
anom_jun_pnw_mean = anom_jun.sel(latitude=pnwlat,longitude=pnwlon).weighted(weights).mean(('latitude','longitude')).compute()


In [4]:
## IMPORT/PREPARE ERA5 DATA 1979–2020 ##


# Import and combine with 2021 data

rawto20 = xr.open_mfdataset(glob.glob(path + 'ERA5*1979_2020*nh.nc'),combine='by_coords',parallel=True, chunks={'latitude': 100, 'longitude': 100, 'time': -1})
rawto20['t2m'] = rawto20.t2m - 273.15
rawto20['t2mland'] = rawto20.t2m.where(landmask>.5)
rawto20['skt'] = rawto20.skt - 273.15
rawto20['sktland'] = rawto20.skt.where(landmask>.5)
rawto20['swvl1'] = rawto20.swvl1.where(landmask>.5)
rawto20['z'] = rawto20.z / 9.81
allraw = xr.concat([rawto20,raw],dim='time')


# Take daily-mean of PNW subset

with pbar:
    pnwdaily = allraw.sel(latitude=pnwlat,longitude=pnwlon)[['sktland','t2mland','z','swvl1']].resample(time='1D').mean('time').compute()
pnwdaily


# Take spatial mean

allraw_pnw_mean = pnwdaily.weighted(weights).mean(('latitude','longitude'))
allraw_df = allraw_pnw_mean.to_dataframe()
allraw_df['day'] = allraw_df.index.strftime('%m-%d')
allraw_df['year'] = allraw_df.index.year


[########################################] | 100% Completed |  9min 56.5s


In [5]:
## IMPORT WAVE ANALYSIS FILE ##

waves = pd.read_csv(path + 'wave_2021.csv', index_col=0)
waves = waves[waves.day>=152][waves.day<=186]
waves['date'] = pd.to_datetime(waves['day']-1, unit='D', origin=pd.Timestamp('2021-01-01'))
waves = waves.set_index('date')


/tmp/ipykernel_12822/1733899153.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  waves = waves[waves.day>=152][waves.day<=186]
/tmp/ipykernel_12822/1733899153.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waves['date'] = pd.to_datetime(waves['day']-1, unit='D', origin=pd.Timestamp('2021-01-01'))
